## Super Data Engineering Take Home Assessment (Ekanshh Praveen)

### 1 - Preparing the dataframe

In [20]:
import pandas as pd
import numpy as np

In [21]:
data = 'Airline Code;DelayTimes;FlightCodes;To_From\nAir Canada (!);[21, 40];20015.0;WAterLoo_NEWYork\n<Air France> (12);[];;Montreal_TORONTO\n(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa\n12. Air France;[78, 66];;Ottawa_VANcouvER\n""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal\n'

# First, we need to split our data that is seperated by \n to individual rows
rows = data.split('\n')
print(rows)

['Airline Code;DelayTimes;FlightCodes;To_From', 'Air Canada (!);[21, 40];20015.0;WAterLoo_NEWYork', '<Air France> (12);[];;Montreal_TORONTO', '(Porter Airways. );[60, 22, 87];20035.0;CALgary_Ottawa', '12. Air France;[78, 66];;Ottawa_VANcouvER', '""".\\.Lufthansa.\\.""";[12, 33];20055.0;london_MONTreal', '']


In [69]:
# As we want to further split the rows into individual cells, we would have to split the row too
cells = []
for i in range(len(rows)):
    cells.append(rows[i].split(";"))

# We would also want to pop the first element of cells to get rid of column names and store them in a seperate list
column_names = cells.pop(0)
cells.pop(len(cells)-1)

#Adding nan for null values
for i in range(len(cells)-1):
    for j in range(len(cells[i])):
        if cells[i][j] == '':
            cells[i][j] = np.nan
            
print("Column Names: ", column_names)
print("Rows: ", cells)

Column Names:  ['Airline Code', 'DelayTimes', 'FlightCodes', 'To_From']
Rows:  [['Air Canada (!)', '[21, 40]', '20015.0', 'WAterLoo_NEWYork'], ['<Air France> (12)', '[]', nan, 'Montreal_TORONTO'], ['(Porter Airways. )', '[60, 22, 87]', '20035.0', 'CALgary_Ottawa'], ['12. Air France', '[78, 66]', nan, 'Ottawa_VANcouvER'], ['""".\\.Lufthansa.\\."""', '[12, 33]', '20055.0', 'london_MONTreal']]


In [70]:
# Now that we have our list of lists we should be able to create our data-frame 

df = pd.DataFrame(cells, columns=column_names)
df.head()
        

,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015.0,WAterLoo_NEWYork
1,<Air France> (12),[],NaN,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035.0,CALgary_Ottawa
3,12. Air France,"[78, 66]",NaN,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055.0,london_MONTreal


### 2 - Populating Flight Codes and Changing column type to int

In [73]:
#First let's get the value of the first row of FlightCodes
flight_code = df.iloc[0]["FlightCodes"]


flight_code = float(flight_code)
flight_code = int(flight_code)
print(flight_code)

# getting the count of rows in the dataframe
row_count = df.shape[0]

# Now let's update the flight codes
# Here we take row_count - 1 to exclude header row
for i in range(row_count):
    df.at[i, 'FlightCodes'] = flight_code
    flight_code += 10

df.head()

20015


,Airline Code,DelayTimes,FlightCodes,To_From
0,Air Canada (!),"[21, 40]",20015,WAterLoo_NEWYork
1,<Air France> (12),[],20025,Montreal_TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary_Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa_VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london_MONTreal


In [76]:
# Now let's change the FlightCodes column to int
df.FlightCodes = df["FlightCodes"].astype(int)

print(df.dtypes)

Airline Code    object
DelayTimes      object
FlightCodes      int32
To_From         object
dtype: object


Now that we have succesfully converted and populated the FlightCodes column, we can move on to splitting the To_From column :)

### 3 - Splitting the To_From Column

In [80]:
# Let's start by first splitting the To_From column
df[['To', 'From']] = df.To_From.str.split("_", expand = True)
df = df.drop('To_From', axis=1)
df.head()

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WAterLoo,NEWYork
1,<Air France> (12),[],20025,Montreal,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALgary,Ottawa
3,12. Air France,"[78, 66]",20045,Ottawa,VANcouvER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,london,MONTreal


In [83]:
# Now, let's convert both the To and From columns to uppercase
df['To'] = df['To'].apply(str.upper)
df['From'] = df['From'].apply(str.upper)
df.head()

,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada (!),"[21, 40]",20015,WATERLOO,NEWYORK
1,<Air France> (12),[],20025,MONTREAL,TORONTO
2,(Porter Airways. ),"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12. Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,""""""".\.Lufthansa.\.""""""","[12, 33]",20055,LONDON,MONTREAL


### 4 - Cleaning Airline Code column

In [94]:
# Let's clean the Airline Code column by removing all punctuations
import string
df['Airline Code'] = df['Airline Code'].str.replace('[{}]'.format(string.punctuation), '', regex=True)
df['Airline Code'] = df['Airline Code'].str.replace('\\', '', regex=True)
df.head()


,Airline Code,DelayTimes,FlightCodes,To,From
0,Air Canada,"[21, 40]",20015,WATERLOO,NEWYORK
1,Air France 12,[],20025,MONTREAL,TORONTO
2,Porter Airways,"[60, 22, 87]",20035,CALGARY,OTTAWA
3,12 Air France,"[78, 66]",20045,OTTAWA,VANCOUVER
4,Lufthansa,"[12, 33]",20055,LONDON,MONTREAL


## Notes and Optimizations

Thank you for reading through my solution! I had a great time solving the assessment :)

Some things I would do if I had to **optimize** my solution:
- Try and use more **vectorized** functions instead of iterating through a dataframe
- Use **PySpark** instead of Pandas as it is significantly quicker

Hope you have a great day!